# Кластеризация и выявление самых частотных слов в каждом кластере

**Задача:** кластеризовать тексты новостей и посмотреть, какие слова являются самыми частотными для каждого кластера.

**Датасет:** 20newsgroups, встроенный датасет из sklearn: https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset


**Используемые технологии:** KMeans для кластеризации, регулярки для токенизации, список стоп-слов из NLTK, некоторые типы данных из collections.

## Загрузка данных

20newsgroups загружается прямо из sklearn, поэтому его загрузка выглядит не так, как загрузка обычного датасета из файла.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
# Bunch - специальный тип данных, применяемый в sklearn.datasets. Вне sklearn мы им пользоваться не будем
type(fetch_20newsgroups())

sklearn.utils._bunch.Bunch

In [ ]:
# Bunch устроен как словарь: по ключам можно достать тексты, имена файлов, названия новостных групп, их номера и описания
# target - числовые значения топиков, удобные для подачи в модель, а target_names - их полные имена
fetch_20newsgroups().keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
# Мы будем кластеризовать тексты. Исходные номера новостных групп сохраним в y на всякий случай, но пользоваться ими не будем.
# Вы можете попробовать разделить тексты на 20 кластеров (это число топиков в датасете) и сравнить ваши кластеры с y
X = fetch_20newsgroups()['data']
y = fetch_20newsgroups()['target']

In [ ]:
X[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [ ]:
y[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Вот такие темы новостей есть в датасете:

In [ ]:
list(enumerate(fetch_20newsgroups()['target_names']))

[(0, 'alt.atheism'),
 (1, 'comp.graphics'),
 (2, 'comp.os.ms-windows.misc'),
 (3, 'comp.sys.ibm.pc.hardware'),
 (4, 'comp.sys.mac.hardware'),
 (5, 'comp.windows.x'),
 (6, 'misc.forsale'),
 (7, 'rec.autos'),
 (8, 'rec.motorcycles'),
 (9, 'rec.sport.baseball'),
 (10, 'rec.sport.hockey'),
 (11, 'sci.crypt'),
 (12, 'sci.electronics'),
 (13, 'sci.med'),
 (14, 'sci.space'),
 (15, 'soc.religion.christian'),
 (16, 'talk.politics.guns'),
 (17, 'talk.politics.mideast'),
 (18, 'talk.politics.misc'),
 (19, 'talk.religion.misc')]

Как видите, если брать только первую часть названия темы (до точки), можно выделить 7 топиков. Давайте попробуем поделить нашу выборку на 7 кластеров.

## Кластеризация

In [ ]:
N_CLUSTERS = 7

In [ ]:
# векторизуем тексты, чтобы привести их в машиночитаемый формат
vectorizer = TfidfVectorizer()

In [ ]:
vectors = vectorizer.fit_transform(X)

In [ ]:
k_means_clusters = KMeans(n_clusters=N_CLUSTERS).fit(vectors)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
k_means_clusters.labels_

array([1, 4, 1, ..., 1, 4, 4], dtype=int32)

Попробуем оценить нашу кластеризацию при помощи коэффициента силуэта. Очевидно, что кластеры новостей разделены плохо: судя по отрицательному значению коэффициента силуэта, темы "наползают" друг на друга. Это в целом ожидаемо: очевидно, что слова из темы "атеизм" могут пересекаться со словами из темы "религия" и т.п., "смешивая" эти топики.

In [ ]:
silhouette_score(vectors, k_means_clusters.labels_)

-0.006532008572238636

## Самые частотные слова

Алгоритм:


*   Делаем словарь, где номеру кластера будет соответствовать список всех принадлежащих ему текстов (воспользуемся defaultdict(list));
*   Для каждого кластера создаем Counter токенов и вносим в него сведеня о количестве токенов во всех текстах;
*   Выводим список частотных слов для каждого кластера.



In [ ]:
from collections import defaultdict, Counter

In [ ]:
import re

In [ ]:
from nltk.corpus import stopwords

Создаем пустой словарь texts_per_cluster, где ключам будут соответствовать списки. Затем мы будем итерироваться по всем лейблам кластеров, которые присвоила текстам модель KMeans. k_means_clusters.labels_ - это список чисел, каждое из которых обозначает, какому кластеру принадлежит соответствующий текст из матрицы X (k_means_clusters.labels_[0] вернет номер кластера для X[0] - первого текста в выборке, k_means_clusters.labels_[i] вернет номер кластера для X[i]).

На каждом шаге цикла ниже мы получаем порядковый номер объекта, который мы рассматриваем в данный момент (i) и номер кластера, в который этот объект входит (label). Мы достаем из множества X текст, соответствующий порядковому номеру i: X[i]. Этот текст нам нужно добавить в список текстов, принадлежащих данному кластеру. Поэтому мы обращаемся к словарю texts_per_cluster по ключу label и добавляем туда текст X[i].

In [ ]:
texts_per_cluster = defaultdict(list)

for i, label in enumerate(k_means_clusters.labels_):
  texts_per_cluster[label].append(X[i])

In [ ]:
# видно, что ключи в словаре соответствуют номерам кластеров
texts_per_cluster.keys()

dict_keys([1, 4, 5, 3, 0, 6, 2])

In [ ]:
# попробуйте достать первые десять текстов первого кластера

In [ ]:
# @title
texts_per_cluster[0][:10]

Итак, для каждого кластера у нас теперь есть списки текстов, которые в него входят. Теперь нам нужно сделать частотный словарь для каждого кластера.

Сделаем функцию, которая принимает на вход все тексты кластера. Функция будет исполнять следующие действия:

1.   Создание пустого объекта Counter (это тип данных, который принимает на вход список элементов и возвращает словарь, где каждому уникальному элементу соответствует количество раз, сколько этот элемент встретился в списке). Это будет частотный список для всего списка текстов.
2.   Далее работаем с каждым текстом. Каждый текст токенизируется и очищается от стоп-слов. Затем мы создаем объект Counter для токенов этого текста, тем самым создавая частотный словарь одного текста. Эти частоты нам нужно записать в общий Counter. Поскольку Counter - это словарь, мы можем работать с ним так же, как с другими словарями в питоне. Поэтому мы воспользуемся методом update(), который будет обновлять наш частотный список для всего списка текстов, пополняя его новыми значениями из каждого конкретного текста.
3.   Вернём частотный список для всего списка текстов.     



In [ ]:
def get_most_frequent_words(list_of_texts):
  c = Counter()

  for text in list_of_texts:
    tokens = re.findall('\w+', text.lower())
    clean_tokens = [token for token in tokens if token not in stopwords.words("english")]
    current_c = Counter(clean_tokens)
    c.update(current_c)

  return c

Нам нужно будет сортировать значения в частотном словаре от большого к малому, поэтому мы немного изменим функцию, которую написали на практике в разделе со звёздочкой. Изменение заключается в добавлении аргумента reverse=True. По умолчанию значения в отсортированном списке идут от меньшего к большему, но этот аргумент изменяет порядок значений.

In [ ]:
def sort_dict_descending(d):
  # сортируем словарь по возрастанию
  return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

Итак, осталось для каждого кластера получить самые частотные слова, отсортировать их и записать в словарь таким образом, чтобы по ключу (номеру кластера) можно было достать отсортированный частотный список слов этого кластера.

Мы снова создадим объект defaultdict(list), чтобы хранить там пары из ключей  - номеров кластеров - и значений: отсортированных частотных списков слов этого кластера. Затем для каждого номера кластера при помощи созданной нами функции get_most_frequent_words получим список слов и их частот. Полученные частотные списки отсортируем и запишем в словарь most_frequent_words_counters по ключу, соответствующему номеру кластера.

In [ ]:
%%time
most_frequent_words_counters = defaultdict(list)

for i in range(N_CLUSTERS):
  print(i)
  word_counts = get_most_frequent_words(texts_per_cluster[i])
  most_frequent_words_counters[i] = list(sort_dict_descending(word_counts).keys())

0
1
2
3
4
5
6
CPU times: user 6min 17s, sys: 54.3 s, total: 7min 11s
Wall time: 7min 42s


Посмотрим на 20 самых частотных слов каждого кластера. Как видно, они довольно шумные, а также сильно пересекаются. Возможно, дальнейшая предобработка текстов могла бы улучшить качество кластеризации.

In [ ]:
for k, v in most_frequent_words_counters.items():
  print(k, v[:20])

0 ['edu', 'team', '2', '1', 'subject', 'game', 'lines', 'organization', 'year', 'writes', 'ca', 'would', '3', 'one', 'article', '0', 'season', 'hockey', '4', 'good']
1 ['edu', 'com', 'lines', 'subject', 'organization', 'x', 'writes', 'article', '1', 'one', 'would', 'like', '2', 'get', 'university', 'posting', 'host', 'use', 'know', 'nntp']
2 ['edu', 'pitt', 'gordon', 'banks', 'geb', 'cs', 'article', 'writes', 'subject', 'organization', 'lines', 'n3jxp', 'skepticism', 'chastity', 'intellect', 'cadre', 'dsl', 'shameful', 'surrender', 'soon']
3 ['com', 'edu', 'writes', 'keith', 'sandvik', 'subject', 'lines', 'article', 'organization', 'morality', 'sgi', 'objective', 'livesey', 'caltech', 'people', 'would', 'one', 'kent', 'apple', 'jon']
4 ['ax', '1', '0', 'edu', '3', '2', 'q', 'w', 'subject', 'lines', 'max', 'r', 'organization', '7', 'p', 'com', 'g', '4', '5', '_']
5 ['one', 'would', 'people', 'edu', 'x', 'god', '1', 'think', 'like', 'subject', 'com', 'know', 'also', 'time', 'us', 'get', 